In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## read data
train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
train.shape , test.shape

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
### Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from  sklearn.model_selection import train_test_split , StratifiedKFold

import warnings 
warnings.simplefilter('ignore')

In [ ]:
#### Get Basic Details of the data files
print('Train data Shape : ' , train.shape)
print('Test data Shape : ' , test.shape)
print('Submission data Shape : ' , submission.shape)

print('#########################################')
print('Null Data Details - Train data')
print(train.isnull().sum())

print('#########################################')
print('Null Data Details - Test data')
print(test.isnull().sum())

In [ ]:
## Descriptive Statistics
train.describe().T

In [ ]:
target = 'pressure'
Id_Cols = ['id' , 'breath_id']
features = [x for x in train.columns if x not in [target]+Id_Cols]

### Lets check the distribution of features
for i in features :
    sns.displot(train[i])
    plt.show()

In [ ]:
# histogran to evaluate target distribution
plt.subplot(1, 2, 1)
train['pressure'].hist(bins=20, density=True)
#plt.ylabel('Number of houses')
plt.xlabel('pressure')

plt.subplot(1, 2, 2)
sns.distplot(train['pressure'])
plt.show()

In [ ]:
#plt.figure(figsize=(10,5))
#plt.subplot(1,4,1)
#sns.distplot(train['pressure'])
#plt.title('Original Distribution')
#
#plt.subplot(1,4,2)
#sns.distplot(np.log(train['pressure']))
#plt.title('Logarithmic')
#
#plt.subplot(1,4,3)
#sns.distplot(1/(train['pressure']))
#plt.title('Reciprocal')
#
#plt.subplot(1,4,4)
#sns.distplot((train['pressure'])**(1/2))
#plt.title('Square Root')
#
#plt.show()

In [ ]:
## Lets find categorical variables
cat_vars = [var for var in train.columns if train[var].dtype == 'O']

## Lets find numerical variables
num_vars = [var for var in train.columns if var not in cat_vars and var != 'pressure']

print('Total Number of categorical Variables : ', len(cat_vars) )
print('-------------------------------------------')
print(cat_vars)
print('Total Number of Numerical Variables : ', len(num_vars) )
print('-------------------------------------------')
print(num_vars)

In [ ]:
# make a list of the variables that contain missing values
vars_with_na = [var for var in train.columns if train[var].isnull().sum() > 0]

# determine percentage of missing values (expressed as decimals)
# and display the result ordered by % of missin data

train[vars_with_na].isnull().mean().sort_values(ascending=False)

In [ ]:
## Temporal variable
# let's explore the values of these temporal variables
train['time_step'].value_counts().sort_index()

In [ ]:
### Lets plot it against pressure
#plt.figure(figsize=(10,5))
#train.groupby('time_step')['pressure'].median().plot()
#plt.ylabel('Median pressure')
#plt.xticks(np.arange(0, 3, 0.01))
#plt.show()

In [ ]:
## Lets plot it against pressure
#plt.figure(figsize=(10,5))
#chk1 = train[round(train['time_step']) == 0]
#chk1.groupby('time_step')['pressure'].median().plot()
#plt.ylabel('Median pressure')
#plt.xticks(np.arange(0, 1, 0.0001))
#plt.show()

In [ ]:
## Lets plot it against pressure
##plt.figure(figsize=(10,5))
##chk2 = train[round(train['time_step']) > 0]
##chk2.groupby('time_step')['pressure'].median().plot()
##plt.ylabel('Median pressure')
##plt.xticks(np.arange(0, 3, 0.0001))
##plt.show()

In [ ]:
#plt.scatter(train['time_step'] , train['pressure'])
#plt.show()

In [ ]:
#plt.scatter(train['R'] , train['pressure'])
#plt.show()

In [ ]:
#plt.scatter(train['C'] , train['pressure'])
#plt.show()

In [ ]:
#plt.scatter(train['u_in'] , train['pressure'])
#plt.show()

In [ ]:
train.groupby('breath_id').size()

In [ ]:
train.groupby('breath_id')['pressure'].mean().plot()

In [ ]:
train[train['breath_id'] == 1].time_step.min()

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
train[train['breath_id'] == 1].groupby('breath_id')['time_step'].plot()
plt.title('time_step for breath id 1')

plt.subplot(1,3,2)
train[train['breath_id'] == 2].groupby('breath_id')['time_step'].plot()
plt.title('time_step for breath id 2')

plt.subplot(1,3,3)
train[train['breath_id'] == 3].groupby('breath_id')['time_step'].plot()
plt.title('time_step for breath id 3')

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
train[train['breath_id'] == 1].groupby('breath_id')['pressure'].plot()
plt.title('pressure for breath id 1')

plt.subplot(1,3,2)
train[train['breath_id'] == 2].groupby('breath_id')['pressure'].plot()
plt.title('pressure for breath id 2')

plt.subplot(1,3,3)
train[train['breath_id'] == 3].groupby('breath_id')['pressure'].plot()
plt.title('pressure for breath id 3')

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
train[train['breath_id'] == 1].groupby(['breath_id' , 'time_step'])['pressure'].plot()
plt.title('pressure for breath id 1')

plt.subplot(1,3,2)
train[train['breath_id'] == 2].groupby(['breath_id' , 'time_step'])['pressure'].plot()
plt.title('pressure for breath id 2')

plt.subplot(1,3,3)
train[train['breath_id'] == 3].groupby(['breath_id' , 'time_step'])['pressure'].plot()
plt.title('pressure for breath id 3')

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,4,1)
train[train['breath_id'] == 1].groupby('breath_id')['R'].plot()
plt.title('R for breath id 1')

plt.subplot(1,4,2)
train[train['breath_id'] == 2].groupby('breath_id')['R'].plot()
plt.title('R for breath id 2')

plt.subplot(1,4,3)
train[train['breath_id'] == 3].groupby('breath_id')['R'].plot()
plt.title('R for breath id 3')

plt.subplot(1,4,4)
train[train['breath_id'] == 4].groupby('breath_id')['R'].plot()
plt.title('R for breath id 4')

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,4,1)
train[train['breath_id'] == 1].groupby('breath_id')['C'].plot()
plt.title('C for breath id 1')

plt.subplot(1,4,2)
train[train['breath_id'] == 2].groupby('breath_id')['C'].plot()
plt.title('C for breath id 2')

plt.subplot(1,4,3)
train[train['breath_id'] == 3].groupby('breath_id')['C'].plot()
plt.title('C for breath id 3')

plt.subplot(1,4,4)
train[train['breath_id'] == 4].groupby('breath_id')['C'].plot()
plt.title('C for breath id 4')

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
train[train['breath_id'] == 1].groupby('breath_id')['u_in'].plot()
plt.title('u_in for breath id 1')

plt.subplot(1,3,2)
train[train['breath_id'] == 2].groupby('breath_id')['u_in'].plot()
plt.title('u_in for breath id 2')

plt.subplot(1,3,3)
train[train['breath_id'] == 3].groupby('breath_id')['u_in'].plot()
plt.title('u_in for breath id 3')

In [ ]:
#sns.catplot(x = 'R' , y='pressure' , hue='C' , kind='swarm' , data=train)
train.R.value_counts()

In [ ]:
train.C.value_counts()

In [ ]:
pd.crosstab(index = train['R'] , columns=train['C']).plot(kind='bar' , figsize=(8,8) , stacked=True)

In [ ]:
train.groupby(['R' , 'C'])['pressure'].median().plot(kind= 'bar' , color=['blue' , 'cyan'])

In [ ]:
#train.groupby(['R' , 'u_out'])['pressure'].mean().plot(kind= 'bar' , color=['blue' , 'cyan'])

plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
train.groupby(['R' , 'u_out'])['pressure'].mean().plot(kind= 'bar' , color=['blue' , 'cyan'])
plt.title('R & u_out')

plt.subplot(1,3,2)
train.groupby(['C' , 'u_out'])['pressure'].mean().plot(kind= 'bar' , color=['blue' , 'cyan'])
plt.title('C and u_out')
plt.subplot(1,3,3)
train.groupby(['R' , 'C'])['pressure'].mean().plot(kind= 'bar' , color=['blue' , 'cyan'])
plt.title('C & R vs Pressure')

In [ ]:
train.groupby(['C' , 'u_out'])['pressure'].mean().plot(kind= 'bar' , color=['blue' , 'cyan'])

In [ ]:
train.head(15)

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
train[train['breath_id'] == 1].groupby('time_step')['u_in'].mean().plot()
plt.title('time_step for breath id 1')

plt.subplot(1,3,2)
train[train['breath_id'] == 2].groupby('time_step')['u_in'].mean().plot()
plt.title('time_step for breath id 2')

plt.subplot(1,3,3)
train[train['breath_id'] == 3].groupby('time_step')['u_in'].mean().plot()
plt.title('time_step for breath id 3')

### Base Model 1

In [ ]:
df_train = train.copy()
df_test = test.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
var = ['R' , 'C' , 'u_in']
df_train[var] = scaler.fit_transform(df_train[var])
df_train.head()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
train.columns 

In [ ]:
vars = [var for var in df_train.columns if var  not in ['id' ,'pressure']]
X = df_train[vars]
y = df_train['pressure']
X.shape , y.shape

In [ ]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
# Define model. Specify a number for random_state to ensure same results each run
model1 = DecisionTreeRegressor(random_state=1)
# Fit model
model1.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = model1.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

In [ ]:
#plt.scatter(val_y, val_predictions)

#### Model 2

In [ ]:
### Hyperparameter tuning to check if it changes the accuracy and control underfitting and overfitting
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [ ]:
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
var = ['R' , 'C' , 'u_in']
df_test[var] = scaler.fit_transform(df_test[var])
df_test.head()

In [ ]:
vars = [var for var in df_test.columns if var  not in ['id']]
model = DecisionTreeRegressor(max_leaf_nodes=5000, random_state=0)
model.fit(train_X, train_y)
preds_test = model.predict(df_test[vars])

In [ ]:
output = pd.DataFrame({'id': df_test.id,
                       'pressure': preds_test})
output.to_csv('submission1.csv', index=False)

In [ ]:
output.head()

***Max leaf node = 5000 is giving better MAE**